In [1]:
# --- Kalshi: live market odds / orderbook (requires credentials) ---
import os, requests
from dotenv import load_dotenv
import datetime
import requests


In [2]:
key_id = "a40ff1c6-12ac-4a6c-9669-ffe12f3de235"

In [3]:
import requests

# Get series information for KXHIGHNY
url = "https://api.elections.kalshi.com/trade-api/v2/series/KXHIGHNY"
response = requests.get(url)
series_data = response.json()

print(f"Series Title: {series_data['series']['title']}")
print(f"Frequency: {series_data['series']['frequency']}")
print(f"Category: {series_data['series']['category']}")

Series Title: Highest temperature in NYC
Frequency: daily
Category: Climate and Weather


In [15]:
# Get all markets for the KXHIGHNY series
markets_url = f"https://api.elections.kalshi.com/trade-api/v2/markets?series_ticker=KXHIGHNY&status=open"
markets_response = requests.get(markets_url)
markets_data = markets_response.json()

print(f"\nActive markets in KXHIGHNY series:")
for market in markets_data['markets']:
    print(f"- {market['ticker']}: {market['title']}")
    print(f"  Event: {market['event_ticker']}")
    print(f"  No ask: {market['no_ask']}¢ | Volume: {market['volume']}")
    print()

# Get details for a specific event if you have its ticker
if markets_data['markets']:
    # Let's get details for the first market's event
    event_ticker = markets_data['markets'][0]['event_ticker']
    event_url = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}"
    event_response = requests.get(event_url)
    event_data = event_response.json()

    print(f"Event Details:")
    print(f"Title: {event_data['event']['title']}")
    print(f"Category: {event_data['event']['category']}")


Active markets in KXHIGHNY series:
- KXHIGHNY-25NOV03-T64: Will the **high temp in NYC** be >64° on Nov 3, 2025?
  Event: KXHIGHNY-25NOV03
  No ask: 100¢ | Volume: 2419

- KXHIGHNY-25NOV03-T57: Will the **high temp in NYC** be <57° on Nov 3, 2025?
  Event: KXHIGHNY-25NOV03
  No ask: 100¢ | Volume: 1689

- KXHIGHNY-25NOV03-B63.5: Will the **high temp in NYC** be 63-64° on Nov 3, 2025?
  Event: KXHIGHNY-25NOV03
  No ask: 100¢ | Volume: 2127

- KXHIGHNY-25NOV03-B61.5: Will the **high temp in NYC** be 61-62° on Nov 3, 2025?
  Event: KXHIGHNY-25NOV03
  No ask: 100¢ | Volume: 9678

- KXHIGHNY-25NOV03-B59.5: Will the **high temp in NYC** be 59-60° on Nov 3, 2025?
  Event: KXHIGHNY-25NOV03
  No ask: 2¢ | Volume: 15803

- KXHIGHNY-25NOV03-B57.5: Will the **high temp in NYC** be 57-58° on Nov 3, 2025?
  Event: KXHIGHNY-25NOV03
  No ask: 100¢ | Volume: 37065

Event Details:
Title: Highest temperature in NYC on Nov 3, 2025?
Category: Climate and Weather


In [18]:
# Get orderbook for a specific market
# Replace with an actual market ticker from the markets list
market_ticker = "KXHIGHNY-25NOV03-B59.5"
orderbook_url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

orderbook_response = requests.get(orderbook_url)
orderbook_data = orderbook_response.json()

print(f"\nOrderbook for {market_ticker}:")
print("YES BIDS:")
for bid in orderbook_data['orderbook']['yes'][:5]:  # Show top 5
    print(f"  Price: {bid[0]}¢, Quantity: {bid[1]}")

print("\nNO BIDS:")
for bid in orderbook_data['orderbook']['no'][:5]:  # Show top 5
    print(f"  Price: {bid[0]}¢, Quantity: {bid[1]}")


Orderbook for KXHIGHNY-25NOV03-B59.5:
YES BIDS:
  Price: 1¢, Quantity: 1250
  Price: 10¢, Quantity: 100
  Price: 19¢, Quantity: 50
  Price: 25¢, Quantity: 800
  Price: 42¢, Quantity: 450

NO BIDS:
  Price: 1¢, Quantity: 1064


In [4]:
import requests
import datetime
import base64
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import padding

# Configuration
API_KEY_ID = "a40ff1c6-12ac-4a6c-9669-ffe12f3de235"
PRIVATE_KEY_PATH = 'key.key'
BASE_URL = "https://api.elections.kalshi.com"

def load_private_key(key_path):
    """Load the private key from file."""
    with open(key_path, "rb") as f:
        return serialization.load_pem_private_key(f.read(), password=None, backend=default_backend())

def create_signature(private_key, timestamp, method, path):
    """Create the request signature."""
    # Strip query parameters before signing
    path_without_query = path.split('?')[0]
    message = f"{timestamp}{method}{path_without_query}".encode('utf-8')
    signature = private_key.sign(
        message,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.DIGEST_LENGTH),
        hashes.SHA256()
    )
    return base64.b64encode(signature).decode('utf-8')

def get(private_key, api_key_id, path, base_url=BASE_URL):
    """Make an authenticated GET request to the Kalshi API."""
    timestamp = str(int(datetime.datetime.now().timestamp() * 1000))
    signature = create_signature(private_key, timestamp, "GET", path)

    headers = {
        'KALSHI-ACCESS-KEY': api_key_id,
        'KALSHI-ACCESS-SIGNATURE': signature,
        'KALSHI-ACCESS-TIMESTAMP': timestamp
    }

    return requests.get(base_url + path, headers=headers)

# Load private key
private_key = load_private_key(PRIVATE_KEY_PATH)

# Get balance
response = get(private_key, API_KEY_ID, "/trade-api/v2/portfolio/balance")
print(f"Your balance: ${response.json()['balance'] / 100:.2f}")

Your balance: $50.63


In [6]:
BASE_URL = "https://api.elections.kalshi.com/trade-api/v2"
MARKET_TICKER = "KXNBAGAME-25NOV24CLETOR-TOR"
response = requests.get(f"{BASE_URL}/markets/{MARKET_TICKER}")
response.raise_for_status()
market_data = response.json()
print(market_data['market']['no_ask']/100)


0.11


In [36]:
print(market_data['market'].keys())

print(market_data['market']['no_ask'])
print(market_data['market']['no_bid'])
print("boobs)")

dict_keys(['can_close_early', 'category', 'close_time', 'custom_strike', 'early_close_condition', 'event_ticker', 'expected_expiration_time', 'expiration_time', 'expiration_value', 'last_price', 'last_price_dollars', 'latest_expiration_time', 'liquidity', 'liquidity_dollars', 'market_type', 'no_ask', 'no_ask_dollars', 'no_bid', 'no_bid_dollars', 'no_sub_title', 'notional_value', 'notional_value_dollars', 'open_interest', 'open_time', 'previous_price', 'previous_price_dollars', 'previous_yes_ask', 'previous_yes_ask_dollars', 'previous_yes_bid', 'previous_yes_bid_dollars', 'price_level_structure', 'price_ranges', 'response_price_units', 'result', 'risk_limit_cents', 'rules_primary', 'rules_secondary', 'settlement_timer_seconds', 'status', 'strike_type', 'subtitle', 'tick_size', 'ticker', 'title', 'volume', 'volume_24h', 'yes_ask', 'yes_ask_dollars', 'yes_bid', 'yes_bid_dollars', 'yes_sub_title'])
15
14
boobs)


In [7]:
import asyncio
import websockets
import json
import base64
import datetime
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend

# ---------- CONFIG ----------
API_KEY_ID = "a40ff1c6-12ac-4a6c-9669-ffe12f3de235"
PRIVATE_KEY_PATH = 'key.key'
MARKET_TICKER = "KXNBAGAME-25NOV24CLETOR-TOR"
WS_URL = "wss://api.elections.kalshi.com/trade-api/ws/v2"  # production websocket
# For testing/demo: wss://demo-api.kalshi.co/trade-api/ws/v2

# ---------- AUTH HELPERS ----------
def load_private_key(key_path):
    with open(key_path, "rb") as f:
        return serialization.load_pem_private_key(f.read(), password=None, backend=default_backend())

def create_signature(private_key, timestamp, method, path):
    message = f"{timestamp}{method}{path}".encode("utf-8")
    signature = private_key.sign(
        message,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.DIGEST_LENGTH),
        hashes.SHA256()
    )
    return base64.b64encode(signature).decode("utf-8")

# ---------- MAIN CONNECTION ----------
async def main():
    private_key = load_private_key(PRIVATE_KEY_PATH)
    timestamp = str(int(datetime.datetime.now().timestamp() * 1000))
    signature = create_signature(private_key, timestamp, "GET", "/trade-api/ws/v2")

    headers = {
        "KALSHI-ACCESS-KEY": API_KEY_ID,
        "KALSHI-ACCESS-SIGNATURE": signature,
        "KALSHI-ACCESS-TIMESTAMP": timestamp,
    }

    async with websockets.connect(WS_URL, additional_headers=headers) as ws:
        print("✅ Connected to Kalshi WebSocket")

        # Subscribe to orderbook updates for your market
        sub_msg = {
            "id": 1,
            "cmd": "subscribe",
            "params": {
                "channels": ["orderbook_delta"],
                "market_tickers": [MARKET_TICKER]
            }
        }
        await ws.send(json.dumps(sub_msg))
        print(f"📡 Subscribed to orderbook updates for {MARKET_TICKER}")

        # Listen for messages forever
        async for msg in ws:
            data = json.loads(msg)
            print(json.dumps(data, indent=2))

await main()



✅ Connected to Kalshi WebSocket
📡 Subscribed to orderbook updates for KXNBAGAME-25NOV24CLETOR-TOR
{
  "type": "subscribed",
  "id": 1,
  "msg": {
    "channel": "orderbook_delta",
    "sid": 1
  }
}
{
  "type": "orderbook_snapshot",
  "sid": 1,
  "seq": 1,
  "msg": {
    "market_ticker": "KXNBAGAME-25NOV24CLETOR-TOR",
    "market_id": "7c185619-72c9-4af2-8396-f4b5216bd7de",
    "yes": [
      [
        1,
        896625
      ],
      [
        2,
        1370
      ],
      [
        3,
        79281
      ],
      [
        4,
        275
      ],
      [
        5,
        47435
      ],
      [
        6,
        50
      ],
      [
        7,
        44590
      ],
      [
        8,
        37
      ],
      [
        9,
        233
      ],
      [
        10,
        105
      ],
      [
        11,
        91480
      ],
      [
        12,
        3792
      ],
      [
        13,
        3987
      ],
      [
        14,
        3485
      ],
      [
        15,
        3073

CancelledError: 

In [14]:
odds_api_key = "63bc135f4d486ac1172d26e3bda95da9"

In [ ]:
SPORT_KEY = "americanfootball_nfl"
BOOKMAKER = "fanduel"
DATE = "2025-11-13"  # game date

url = "https://api.the-odds-api.com/v4/sports/{}/odds".format(SPORT_KEY)
params = {
    "regions": "us",
    "markets": "h2h",
    "dateFormat": "iso",
    "oddsFormat": "decimal",
    "apiKey": odds_api_key
}
resp = requests.get(url, params=params)
resp.raise_for_status()
data = resp.json()

for game in data:
    print(game["home_team"], "vs", game["away_team"])
    for b in game["bookmakers"]:
        if b["key"] == BOOKMAKER:
            for market in b["markets"]:
                if market["key"] == "h2h":
                    for outcome in market["outcomes"]:
                        print(outcome["name"], outcome["price"])
    print("-----")

New England Patriots vs New York Jets
New England Patriots 1.15
New York Jets 5.2
-----
Miami Dolphins vs Washington Commanders
Miami Dolphins 1.7
Washington Commanders 2.2
-----
Atlanta Falcons vs Carolina Panthers
Atlanta Falcons 1.51
Carolina Panthers 2.64
-----
Buffalo Bills vs Tampa Bay Buccaneers
Buffalo Bills 1.37
Tampa Bay Buccaneers 3.2
-----
Minnesota Vikings vs Chicago Bears
Chicago Bears 2.3
Minnesota Vikings 1.65
-----
Pittsburgh Steelers vs Cincinnati Bengals
Cincinnati Bengals 3.05
Pittsburgh Steelers 1.4
-----
New York Giants vs Green Bay Packers
Green Bay Packers 1.28
New York Giants 3.9
-----
Tennessee Titans vs Houston Texans
Houston Texans 1.36
Tennessee Titans 3.25
-----
Jacksonville Jaguars vs Los Angeles Chargers
Jacksonville Jaguars 2.32
Los Angeles Chargers 1.64
-----
Arizona Cardinals vs San Francisco 49ers
Arizona Cardinals 2.38
San Francisco 49ers 1.61
-----
Los Angeles Rams vs Seattle Seahawks
Los Angeles Rams 1.61
Seattle Seahawks 2.38
-----
Cleveland Brow

In [27]:
import pandas as pd

In [26]:
pd.DataFrame(data[0])

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmakers
0,32ec2bd4ac9e4c40f9f5cbed7e67f837,americanfootball_nfl,NFL,2025-11-14T01:16:41Z,New England Patriots,New York Jets,"{'key': 'draftkings', 'title': 'DraftKings', '..."
1,32ec2bd4ac9e4c40f9f5cbed7e67f837,americanfootball_nfl,NFL,2025-11-14T01:16:41Z,New England Patriots,New York Jets,"{'key': 'fanduel', 'title': 'FanDuel', 'last_u..."
2,32ec2bd4ac9e4c40f9f5cbed7e67f837,americanfootball_nfl,NFL,2025-11-14T01:16:41Z,New England Patriots,New York Jets,"{'key': 'mybookieag', 'title': 'MyBookie.ag', ..."
3,32ec2bd4ac9e4c40f9f5cbed7e67f837,americanfootball_nfl,NFL,2025-11-14T01:16:41Z,New England Patriots,New York Jets,"{'key': 'betus', 'title': 'BetUS', 'last_updat..."
4,32ec2bd4ac9e4c40f9f5cbed7e67f837,americanfootball_nfl,NFL,2025-11-14T01:16:41Z,New England Patriots,New York Jets,"{'key': 'lowvig', 'title': 'LowVig.ag', 'last_..."
5,32ec2bd4ac9e4c40f9f5cbed7e67f837,americanfootball_nfl,NFL,2025-11-14T01:16:41Z,New England Patriots,New York Jets,"{'key': 'betonlineag', 'title': 'BetOnline.ag'..."
6,32ec2bd4ac9e4c40f9f5cbed7e67f837,americanfootball_nfl,NFL,2025-11-14T01:16:41Z,New England Patriots,New York Jets,"{'key': 'betrivers', 'title': 'BetRivers', 'la..."
7,32ec2bd4ac9e4c40f9f5cbed7e67f837,americanfootball_nfl,NFL,2025-11-14T01:16:41Z,New England Patriots,New York Jets,"{'key': 'bovada', 'title': 'Bovada', 'last_upd..."


In [ ]:

BASE_URL = "https://api.elections.kalshi.com/trade-api/v2"
MARKET_TICKER = "KXNFLGAME-25NOV13NYJNE-NE"
EVENT_ID = "32ec2bd4ac9e4c40f9f5cbed7e67f837"  # replace with real one

In [ ]:

url = f"https://api.the-odds-api.com/v4/sports/{SPORT_KEY}/events/{EVENT_ID}/odds"
params = {
    "apiKey": odds_api_key,
    "regions": "us",
    "markets": "h2h",
    "oddsFormat": "decimal"
}
r = requests.get(url, params=params)
r.raise_for_status()
odds = r.json()

for b in odds["bookmakers"]:
    if b["key"] == "fanduel":
        for market in b["markets"]:
            if market["key"] == "h2h":
                print("FanDuel odds (de-vigged):")

                # Extract implied probs
                outcomes = market["outcomes"]
                probs = []
                for o in outcomes:
                    prob = 1 / o["price"]
                    probs.append({"name": o["name"], "prob": prob})

                # Compute overround
                total_prob = sum(p["prob"] for p in probs)
                overround = total_prob - 1

                # Remove half the overround from each side
                for p in probs:
                    fair_prob = p["prob"] - (overround / 2)
                    fair_prob = max(fair_prob, 0)  # sanity floor
                    fair_decimal = 1 / fair_prob if fair_prob > 0 else None
                    print(f"  {p['name']}: {fair_prob:.4f}")

                # Optionally also print the raw implieds
                print("\nRaw implied probabilities:")
                for p in probs:
                    print(f"  {p['name']}: {p['prob']:.4f}")


print("KALSHI ODDS:::")
response = requests.get(f"{BASE_URL}/markets/{MARKET_TICKER}")
response.raise_for_status()
market_data = response.json()
print("PATS ODDS", market_data['market']['yes_ask']/100)
print("JETS ODDS", market_data['market']['no_ask']/100)


FanDuel odds (de-vigged):
  New England Patriots: 0.8115  (fair decimal: 1.232)
  New York Jets: 0.1885  (fair decimal: 5.304)

Raw implied probabilities:
  New England Patriots: 0.8403
  New York Jets: 0.2174
KALSHI ODDS:::
JETS ODDS 0.21
PATS ODDS 0.8


In [53]:
import requests

# --- FanDuel (via The Odds API): de-vig h2h ---
url = f"https://api.the-odds-api.com/v4/sports/{SPORT_KEY}/events/{EVENT_ID}/odds"
params = {
    "apiKey": odds_api_key,
    "regions": "us",
    "markets": "h2h",
    "oddsFormat": "decimal"
}
r = requests.get(url, params=params)
r.raise_for_status()
odds = r.json()

# Extract FanDuel two-way market and de-vig
fd_fair = {}  # {"NE": prob, "NYJ": prob}

def normalize_team(name: str) -> str:
    n = name.lower()
    if "patriot" in n or "new england" in n or "ne patriots" in n:
        return "NE"
    if "jet" in n or "new york jets" in n or "ny jets" in n or "nyj" in n:
        return "NYJ"
    return name  # fallback

for b in odds.get("bookmakers", []):
    if b.get("key") == "fanduel":
        for market in b.get("markets", []):
            if market.get("key") == "h2h":
                outcomes = market.get("outcomes", [])
                # raw implied probs
                implied = []
                for o in outcomes:
                    name = normalize_team(o["name"])
                    p = 1.0 / float(o["price"])
                    implied.append((name, p))
                if len(implied) != 2:
                    continue  # ensure exactly 2 outcomes

                total_prob = sum(p for _, p in implied)
                overround = total_prob - 1.0
                # subtract half the overround from each side
                for name, p in implied:
                    fair_p = max(p - overround / 2.0, 0.0)
                    fd_fair[name] = fair_p
                break

# --- Kalshi market: implied probs from ask prices ---
print("KALSHI ODDS:::")
response = requests.get(f"{BASE_URL}/markets/{MARKET_TICKER}")
response.raise_for_status()
market_data = response.json()["market"]

# For market ...-NE : YES = Patriots, NO = Jets
kalshi_ne_prob = float(market_data["yes_ask"]) / 100.0 if market_data.get("yes_ask") is not None else None
kalshi_nyj_prob = float(market_data["no_ask"]) / 100.0  if market_data.get("no_ask")  is not None else None

print("Patriots ask (prob):", kalshi_ne_prob)
print("Jets ask (prob):", kalshi_nyj_prob)

# --- Compare: (Kalshi − FanDuel_fair) for each side ---
fd_ne = fd_fair.get("NE")
fd_nyj = fd_fair.get("NYJ")

print("\nFanDuel de-vigged probabilities:")
print("Patriots (NE):", fd_ne)
print("Jets (NYJ):", fd_nyj)

print("\nDifferences (Kalshi − FanDuel_fair):")
if kalshi_ne_prob is not None and fd_ne is not None:
    diff_ne = kalshi_ne_prob - fd_ne
    print(f"NE:  {diff_ne:+.4f}")
else:
    print("NE:  N/A")

if kalshi_nyj_prob is not None and fd_nyj is not None:
    diff_nyj = kalshi_nyj_prob - fd_nyj
    print(f"NYJ: {diff_nyj:+.4f}")
else:
    print("NYJ: N/A")

# Optional: quick read of which book is higher implied for each side
if kalshi_ne_prob is not None and fd_ne is not None:
    higher_ne = "Kalshi" if kalshi_ne_prob > fd_ne else "FanDuel"
    print(f"\nHigher implied on NE: {higher_ne}")
if kalshi_nyj_prob is not None and fd_nyj is not None:
    higher_nyj = "Kalshi" if kalshi_nyj_prob > fd_nyj else "FanDuel"
    print(f"Higher implied on NYJ: {higher_nyj}")


KALSHI ODDS:::
Patriots ask (prob): 0.93
Jets ask (prob): 0.09

FanDuel de-vigged probabilities:
Patriots (NE): 0.9292228390166534
Jets (NYJ): 0.07077716098334656

Differences (Kalshi − FanDuel_fair):
NE:  +0.0008
NYJ: +0.0192

Higher implied on NE: Kalshi
Higher implied on NYJ: Kalshi
